## Android app using Torch Model

ref: https://pytorch.org/mobile/android/#quickstart-with-a-helloworld-example  
(image segmentation https://pytorch.org/tutorials/beginner/deeplabv3_on_android.html)

my repository: https://github.com/saugkim/TorchIntro/tree/main/Android/mobilenet

project created 16.1.2022  
Android studio IDE version: 4.2.1      
org.pytorch:pytorch_android:1.10.0  
minimum SDK version: 21  

<br>

build warning: possible issues!  <br>
INSTALL_FAILED_INSUFFICIENT_STORAGE running on emulator (Nexus API 28)   
The device needs more free storage to install the application (extra space is needed in addition to APK size).  
  
temp solution: wipe data AVD every time.....

true solution: increase internal storage of android virtual device see below  

   - press `Show Advanced Settings` button and scroll down to `Memory and Storage` part  
   - change value of internal storage (default 800, Nexus 4 API 28) to 1500  
   

<img src="https://raw.githubusercontent.com/saugkim/TorchIntro/main/Android/avd_setting1.PNG" width=600 />
<img src="https://raw.githubusercontent.com/saugkim/TorchIntro/main/Android/avd_setting2.PNG" width=600 />

### step 1. conver PyTorch model to TorchScript format to use in android app

In [6]:
import torch
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile

model = torchvision.models.mobilenet_v2(pretrained=True)
model.eval()
#example = torch.rand(1, 3, 224, 224)
#traced_script_module = torch.jit.trace(model, example)
#traced_script_module_optimized = optimize_for_mobile(traced_script_module)
#traced_script_module_optimized._save_for_lite_interpreter("app/src/main/assets/model.ptl")
scriptedm = torch.jit.script(model)
torch.jit.save(scriptedm, "mobilenet_scripted.pt")

###  Image Segmentation DeepLabV3 on Android

In [ ]:
import torch
from PIL import Image
from torchvision import transforms

# use deeplabv3_resnet50 instead of resnet101 to reduce the model size
model = torch.hub.load('pytorch/vision:v0.7.0', 'deeplabv3_resnet50', pretrained=True)
model.eval()

scriptedm = torch.jit.script(model)
torch.jit.save(scriptedm, "deeplabv3_scripted.pt")


example_image = "https://raw.githubusercontent.com/jeffxtang/android-demo-app/new_demo_apps/ImageSegmentation/app/src/main/assets/deeplab.jpg"

input_image = Image.open("deeplab.jpg")
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)
with torch.no_grad():
    output = model(input_batch)['out'][0]

print(input_batch.shape)
print(output.shape)


#output shoule be below:
#
#torch.Size([1, 3, 400, 400])
#torch.Size([21, 400, 400])

### step 2. Build a new Android app and add scripted torch Model and PyTorch Library on Android


Saved model file *model_name.pt* to project’s assets folder.  

        path to add model: app/src/main/assets/model_name.pt

<br>

Open build.gradle module file, and add the following two libraries in dependencies    


*build.gradle(Module: appname.app)*  
midSdkVersion 21 for torch 1.10.0 required  

```    
    
    android {  
        compileSdkVersion 31
    
        defaultConfig {
            applicationId "xxx.xxx.xxx"
            minSdkVersion 21
            targetSdkVersion 31
            versionCode 1
            versionName "1.0"
        }
    }
    
    dependencies {  
        implementation 'org.pytorch:pytorch_android:1.10.0'   
        implementation 'org.pytorch:pytorch_android_torchvision:1.10.0'   
        ...  
    } 
```
   

*build.gradle(Project: appname)*  
   **jcenter() repository** cannot be removed at this moment - build error if removed:
    
```
    allproject {
        repositories {
            google()
            mavenCentral()
            jcenter() // Warning: this repository is going to shut down soon
        }
    }
```

### step 3. process model input and run model with input and get output

All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H and W are expected to be at least 224. The images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]

In [ ]:
Inside onCreate() of MainActivity.java

    # preparing input tensor
    final Tensor inputTensor = TensorImageUtils.bitmapToFloat32Tensor(bitmap,
        TensorImageUtils.TORCHVISION_NORM_MEAN_RGB, 
        TensorImageUtils.TORCHVISION_NORM_STD_RGB, 
        MemoryFormat.CHANNELS_LAST);

    
    ## RUN Interface
    # running the model
    final Tensor outputTensor = module.forward(IValue.from(inputTensor)).toTensor();
    # getting tensor content as java array of floats
    final float[] scores = outputTensor.getDataAsFloatArray();

    
    ## RESULT 
    # searching for the index with maximum score
    float maxScore = -Float.MAX_VALUE;
    int maxScoreIdx = -1;
    for (int i = 0; i < scores.length; i++) {
      if (scores[i] > maxScore) {
        maxScore = scores[i];
        maxScoreIdx = i;
      }
    }

    String className = ImageNetClasses.IMAGENET_CLASSES[maxScoreIdx];